# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)




1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [2]:
%%capture
!pip install dlt[duckdb] # Install dlt with all the necessary DuckDB dependencies

In [10]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
Pipeline dlt_colab_kernel_launcher load step completed in 0.18 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708059555.7720847 is LOADED and contains no failed jobs
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'Cit

In [14]:
import dlt
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')
info = generators_pipeline.run(people_1(), table_name='people', write_disposition='replace', primary_key='ID')

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.32 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708059611.2757783 is LOADED and contains no failed jobs


In [15]:
info = generators_pipeline.run(people_2(), table_name='people', write_disposition='append')
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.17 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708059615.0792572 is LOADED and contains no failed jobs


In [16]:
import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

people_df = conn.sql("SELECT * FROM people").df()
people_df.head(10)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708059611.2757783,OB5gfyTGViWfSg,None
1,2,Person_2,27,City_A,1708059611.2757783,DTYEBXDzoKkHjw,None
2,3,Person_3,28,City_A,1708059611.2757783,LO9zBM//Q0NA+w,None
3,4,Person_4,29,City_A,1708059611.2757783,8NkjxAhaaltRWw,None
4,5,Person_5,30,City_A,1708059611.2757783,v2hdK5l2YKFprg,None
5,3,Person_3,33,City_B,1708059615.0792572,YYxvWQvRjKyl3g,Job_3
6,4,Person_4,34,City_B,1708059615.0792572,/lzYfWZTJNSGBg,Job_4
7,5,Person_5,35,City_B,1708059615.0792572,U8i2tKdeeFFRcw,Job_5
8,6,Person_6,36,City_B,1708059615.0792572,N2N2JcQP6hlCrA,Job_6
9,7,Person_7,37,City_B,1708059615.0792572,+CjpOBO8WYHL5g,Job_7


In [13]:
conn.sql("select sum(age) from people").df().head()

,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [18]:
import dlt
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')
info = generators_pipeline.run(people_1(), table_name='people', write_disposition='replace', primary_key='ID')

print(info)
info = generators_pipeline.run(people_2(), table_name='people', write_disposition='merge',primary_key='ID')
print(info)

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

people_df = conn.sql("SELECT * FROM people").df()
people_df.head(10)

Pipeline dlt_colab_kernel_launcher load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708059645.358649 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.34 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708059646.1605506 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708059645.358649,vuiozO+vJX7E0g,None
1,2,Person_2,27,City_A,1708059645.358649,8Eq7c3WwPdKp8Q,None
2,5,Person_5,35,City_B,1708059646.1605506,tLUD6iQwmjIRzw,Job_5
3,8,Person_8,38,City_B,1708059646.1605506,SSjmGFmEgKZaFg,Job_8
4,7,Person_7,37,City_B,1708059646.1605506,XikAAo8wqt4INQ,Job_7
5,4,Person_4,34,City_B,1708059646.1605506,73PIttBnbuqLmg,Job_4
6,3,Person_3,33,City_B,1708059646.1605506,DS+P7WzofC3/mg,Job_3
7,6,Person_6,36,City_B,1708059646.1605506,l0aRlVFjZ/KwGg,Job_6


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX